ETL: Carga Areas de Servicio

In [41]:
# Imports 
from pyspark.sql.types import IntegerType, StringType, DateType, LongType
from pyspark.sql import functions as f, SparkSession, types as t
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.functions import col, length

In [42]:
class MySQLConnector:
    def __init__(self, spark: SparkSession, connection_properties: dict, url: str):
        self.spark = spark
        self.properties = connection_properties
        self.url = url

    def get_dataframe(self, sql_query: str):        
        df = self.spark.read.jdbc(
            url=self.url,
            table=sql_query,
            properties=self.properties
        )
        return df
    
    def save_db(self, df, tabla):
        df.write.jdbc(
            url=self.url,
            table=tabla,
            mode='append',
            properties=self.properties
        )
        
def create_spark_session(path_jar_driver):    
    conf = SparkConf().set('spark.driver.extraClassPath', path_jar_driver)
    spark_context = SparkContext(conf=conf)
    sql_context = SQLContext(spark_context)
    return sql_context.sparkSession  

def get_dataframe_from_csv(_PATH, _sep):
    return spark.read.load(_PATH, format="csv", sep=_sep, inferSchema="true", header='true')

In [43]:
# LLENAR CON EL USUARIO DE CADA UNO
db_user = 'Estudiante_58_202415'
db_psswd = 'Estudiante_202425426'


connection_properties = {
    "user": db_user,
    "password": db_psswd,
    "driver": "com.mysql.cj.jdbc.Driver"
}

source_db_string_connection = 'jdbc:mysql://157.253.236.120:8080/RaSaTransaccional_ETL'
destination_db_string_connection = f'jdbc:mysql://157.253.236.120:8080/{db_user}'

# Driver de conexion
# LINUX
#path_jar_driver = '/opt/mysql/lib/mysql-connector-java-8.0.28.jar'
# WINDOWS
path_jar_driver = 'C:\Program Files (x86)\MySQL\Connector J 8.0\mysql-connector-java-8.0.28.jar'

In [44]:
spark = create_spark_session(path_jar_driver)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at C:\Users\estudiante\AppData\Local\Temp\ipykernel_11104\2555830562.py:25 

In [45]:
conn_orig = MySQLConnector(spark=spark, connection_properties=connection_properties, url=source_db_string_connection)
conn_dest = MySQLConnector(spark=spark, connection_properties=connection_properties, url=destination_db_string_connection)

## Proceso de ETL para las dimensiones `TiposBeneficio` y `MiniCondicionesTipoBeneficio`

![Modelo Movimientos](./images/CondicionesTipoBeneficio.png)

## Dimensión Tipo Beneficio

### Extraction

In [9]:
# Se extraen los datos y se muestran
sql_tipo_beneficio = '''(SELECT DISTINCT IdTipoBeneficio_T, Nombre FROM RaSaTransaccional_ETL.FuenteTiposBeneficio_ETL) AS Beneficios'''
tipo_beneficio = conn_orig.get_dataframe(sql_tipo_beneficio)
tipo_beneficio.show()

+-----------------+--------------------+
|IdTipoBeneficio_T|              Nombre|
+-----------------+--------------------+
|                5|Abortion For Whic...|
|               10|   Accidental Dental|
|               20|Adult Frames And ...|
|               25|  Allergy Injections|
|               30|     Allergy Testing|
|               40|          Anesthesia|
|               45|Anesthesia Servic...|
|               50|Applied Behavior ...|
|               60|Autism Spectrum D...|
|               65|Autism Spectrum D...|
|               70|Autism Spectrum D...|
|               75|   Bariatric Surgery|
|               80|Basic Dental Care...|
|               85|Basic Dental Care...|
|               90|Blood And Blood S...|
|               95| Bone Marrow Testing|
|              100|Bone Marrow Trans...|
|              105|        Brain Injury|
|              110|Breast Implant Re...|
|              115|   Cancer Treatments|
+-----------------+--------------------+
only showing top

### Transformation

In [10]:
# Se incluye el indice de la bodega de datos y se ordenan las columnas en el dataframe
tipo_beneficio = tipo_beneficio.coalesce(1).withColumn('IdTipoBeneficio_DWH', f.monotonically_increasing_id() + 1)
tipo_beneficio = tipo_beneficio.select('IdTipoBeneficio_DWH','IdTipoBeneficio_T','Nombre')
tipo_beneficio.show()

+-------------------+-----------------+--------------------+
|IdTipoBeneficio_DWH|IdTipoBeneficio_T|              Nombre|
+-------------------+-----------------+--------------------+
|                  1|                5|Abortion For Whic...|
|                  2|               10|   Accidental Dental|
|                  3|               20|Adult Frames And ...|
|                  4|               25|  Allergy Injections|
|                  5|               30|     Allergy Testing|
|                  6|               40|          Anesthesia|
|                  7|               45|Anesthesia Servic...|
|                  8|               50|Applied Behavior ...|
|                  9|               60|Autism Spectrum D...|
|                 10|               65|Autism Spectrum D...|
|                 11|               70|Autism Spectrum D...|
|                 12|               75|   Bariatric Surgery|
|                 13|               80|Basic Dental Care...|
|                 14|   

### Load

In [11]:
# Se realiza la carga delos datos
conn_dest.save_db(tipo_beneficio, 'Estudiante_58_202415.Rs_TiposBeneficio')

## Dimensión MiniCondiciones Tipo de Servicio

### Extraction

In [31]:
# Se extraen los datos y se muestran
sql_mini_condiciones = '''(SELECT DISTINCT IdTipoBeneficio_T, EstaCubiertaPorSeguro as EstaCubiertoPorSeguro, EsEHB, TieneLimiteCuantitativo, ExcluidoDelDesembolsoMaximoDentroDeLaRed, ExcluidoDelDesembolsoMaximoFueraDeLaRed, UnidadDelLimite, Fecha as Annio FROM RaSaTransaccional_ETL.FuenteTiposBeneficio_ETL) AS Condiciones'''
mini_condiciones = conn_orig.get_dataframe(sql_mini_condiciones)
mini_condiciones.show()

+-----------------+---------------------+-----+-----------------------+----------------------------------------+---------------------------------------+--------------------+-----+
|IdTipoBeneficio_T|EstaCubiertoPorSeguro|EsEHB|TieneLimiteCuantitativo|ExcluidoDelDesembolsoMaximoDentroDeLaRed|ExcluidoDelDesembolsoMaximoFueraDeLaRed|     UnidadDelLimite|Annio|
+-----------------+---------------------+-----+-----------------------+----------------------------------------+---------------------------------------+--------------------+-----+
|                5|                   No|   No|                     No|                                      No|                                     No|                    | 2017|
|                5|                  Yes|   No|                     No|                                      No|                                    Yes|                    | 2020|
|               10|                  Yes|  Yes|                    Yes|                             

### Transformation

In [32]:
# Se incluye el indice de la bodega de datos y se ordenan las columnas en el dataframe
mini_condiciones = mini_condiciones.coalesce(1).withColumn('IdCondicionesBeneficios_DWH', f.monotonically_increasing_id() + 1)
mini_condiciones = mini_condiciones.select('IdCondicionesBeneficios_DWH','IdTipoBeneficio_T','EstaCubiertoPorSeguro','EsEHB','TieneLimiteCuantitativo','ExcluidoDelDesembolsoMaximoDentroDeLaRed','ExcluidoDelDesembolsoMaximoFueraDeLaRed','UnidadDelLimite','Annio')
mini_condiciones.show()

+---------------------------+-----------------+---------------------+-----+-----------------------+----------------------------------------+---------------------------------------+--------------------+-----+
|IdCondicionesBeneficios_DWH|IdTipoBeneficio_T|EstaCubiertoPorSeguro|EsEHB|TieneLimiteCuantitativo|ExcluidoDelDesembolsoMaximoDentroDeLaRed|ExcluidoDelDesembolsoMaximoFueraDeLaRed|     UnidadDelLimite|Annio|
+---------------------------+-----------------+---------------------+-----+-----------------------+----------------------------------------+---------------------------------------+--------------------+-----+
|                          1|                5|                   No|   No|                     No|                                      No|                                     No|                    | 2017|
|                          2|                5|                  Yes|   No|                     No|                                      No|                            

In [33]:
# Se exploran los datos de las columnas categoricas de tipo 'Yes/No' para verificar que solo existan este tipo de valores
mini_condiciones.groupby('EstaCubiertoPorSeguro').count().orderBy("count", ascending=False).show()
mini_condiciones.groupby('EsEHB').count().orderBy("count", ascending=False).show()
mini_condiciones.groupby('TieneLimiteCuantitativo').count().orderBy("count", ascending=False).show()
mini_condiciones.groupby('ExcluidoDelDesembolsoMaximoDentroDeLaRed').count().orderBy("count", ascending=False).show()
mini_condiciones.groupby('ExcluidoDelDesembolsoMaximoFueraDeLaRed').count().orderBy("count", ascending=False).show()

+---------------------+-----+
|EstaCubiertoPorSeguro|count|
+---------------------+-----+
|                  Yes|  313|
|                   No|   30|
|                False|    2|
+---------------------+-----+

+-----+-----+
|EsEHB|count|
+-----+-----+
|   No|  225|
|  Yes|  103|
| True|   17|
+-----+-----+

+-----------------------+-----+
|TieneLimiteCuantitativo|count|
+-----------------------+-----+
|                     No|  216|
|                    Yes|  119|
|                     Si|   10|
+-----------------------+-----+

+----------------------------------------+-----+
|ExcluidoDelDesembolsoMaximoDentroDeLaRed|count|
+----------------------------------------+-----+
|                                      No|  327|
|                                     Yes|   18|
+----------------------------------------+-----+

+---------------------------------------+-----+
|ExcluidoDelDesembolsoMaximoFueraDeLaRed|count|
+---------------------------------------+-----+
|                         

Dado que existen valores diferentes a `Yes` y `No` se deben hacer transformaciones adicionales para estandarizar los datos.

In [34]:
# Se define una función que transforme los datos 'Si' y 'True' a valores 'Yes'; y los datos 'False' a valores 'No'
def transformar_valores(df, nombre_columna):
    return df.withColumn(
        nombre_columna,
        f.when(f.col(nombre_columna).isin('Si', 'True'), 'Yes') \
         .when(f.col(nombre_columna) == 'False', 'No') \
         .otherwise(f.col(nombre_columna))
    )

# Se especifican ahora las columnas que se deben transformar segun la mini-exploración anterior
columnas_a_transformar = ['EstaCubiertoPorSeguro', 'EsEHB', 'TieneLimiteCuantitativo']

# Se realiza la transformación de las columnas
for col in columnas_a_transformar:
    mini_condiciones = transformar_valores(mini_condiciones, col)
    
# Se inspecciona el dataframe
mini_condiciones.show()

+---------------------------+-----------------+---------------------+-----+-----------------------+----------------------------------------+---------------------------------------+--------------------+-----+
|IdCondicionesBeneficios_DWH|IdTipoBeneficio_T|EstaCubiertoPorSeguro|EsEHB|TieneLimiteCuantitativo|ExcluidoDelDesembolsoMaximoDentroDeLaRed|ExcluidoDelDesembolsoMaximoFueraDeLaRed|     UnidadDelLimite|Annio|
+---------------------------+-----------------+---------------------+-----+-----------------------+----------------------------------------+---------------------------------------+--------------------+-----+
|                          1|                5|                   No|   No|                     No|                                      No|                                     No|                    | 2017|
|                          2|                5|                  Yes|   No|                     No|                                      No|                            

In [35]:
# Se exploran nuevamente los datos de las columnas categoricas de tipo 'Yes/No' para verificar que solo existan este tipo de valores
mini_condiciones.groupby('EstaCubiertoPorSeguro').count().orderBy("count", ascending=False).show()
mini_condiciones.groupby('EsEHB').count().orderBy("count", ascending=False).show()
mini_condiciones.groupby('TieneLimiteCuantitativo').count().orderBy("count", ascending=False).show()
mini_condiciones.groupby('ExcluidoDelDesembolsoMaximoDentroDeLaRed').count().orderBy("count", ascending=False).show()
mini_condiciones.groupby('ExcluidoDelDesembolsoMaximoFueraDeLaRed').count().orderBy("count", ascending=False).show()

+---------------------+-----+
|EstaCubiertoPorSeguro|count|
+---------------------+-----+
|                  Yes|  313|
|                   No|   32|
+---------------------+-----+

+-----+-----+
|EsEHB|count|
+-----+-----+
|   No|  225|
|  Yes|  120|
+-----+-----+

+-----------------------+-----+
|TieneLimiteCuantitativo|count|
+-----------------------+-----+
|                     No|  216|
|                    Yes|  129|
+-----------------------+-----+

+----------------------------------------+-----+
|ExcluidoDelDesembolsoMaximoDentroDeLaRed|count|
+----------------------------------------+-----+
|                                      No|  327|
|                                     Yes|   18|
+----------------------------------------+-----+

+---------------------------------------+-----+
|ExcluidoDelDesembolsoMaximoFueraDeLaRed|count|
+---------------------------------------+-----+
|                                     No|  236|
|                                    Yes|  109|
+-----

In [39]:
# Por ultimo se verifica que la columna Annio no contenga nulos y se verifica el tipo de dato
mini_condiciones.groupby('Annio').count().orderBy("count", ascending=False).show()
mini_condiciones.printSchema()

+-----+-----+
|Annio|count|
+-----+-----+
| 2017|  155|
| 2018|   62|
| 2021|   58|
| 2019|   37|
| 2020|   33|
+-----+-----+

root
 |-- IdCondicionesBeneficios_DWH: long (nullable = false)
 |-- IdTipoBeneficio_T: integer (nullable = true)
 |-- EstaCubiertoPorSeguro: string (nullable = true)
 |-- EsEHB: string (nullable = true)
 |-- TieneLimiteCuantitativo: string (nullable = true)
 |-- ExcluidoDelDesembolsoMaximoDentroDeLaRed: string (nullable = true)
 |-- ExcluidoDelDesembolsoMaximoFueraDeLaRed: string (nullable = true)
 |-- UnidadDelLimite: string (nullable = true)
 |-- Annio: integer (nullable = true)



### Load

In [47]:
# Se realiza la carga delos datos
conn_dest.save_db(mini_condiciones, 'Estudiante_58_202415.Rs_MiniCondicionesTipoBeneficio')